In [44]:
import pandas as pd
import pymysql
import numpy as np
import requests
from datetime import datetime, timedelta
import time
from pycoingecko import CoinGeckoAPI

# CoinMarketCap.com API Key
api_key = "a86e3326-8c8d-434e-897a-5d6becbf1f35"
top_symbols_count = 1300
required_tags = ['Binance Smart Chain Ecosystem', 'Polkadot Ecosystem', 'Cosmos Ecosystem', 'Solana Ecosystem', 'Avalanche Ecosystem', 'HECO Chain Ecosystem', 'Layer 1', 'Decentralized Finance (DeFi)', 'Yield Farming', 'Decentralized Exchange Token (DEX)', 'Governance', 'DaoMaker Ecosystem', 'Gaming', 'Stablecoins', 'Asset Manager', 'Centralized Exchange Token (CEX)', 'Eth 2.0 Staking', 'Yield Aggregator', 'Lending/Borrowing', 'Options', 'Launchpad']
cg = CoinGeckoAPI()

In [45]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

### Today's Top Coins (marcketCap) under 1100

In [46]:
def get_market_data(page):
    market_data = cg.get_coins_markets(vs_currency="usd", page=page, per_page=250)
    ids = []
    for data in market_data:
        roi = data['roi']['times'] if data['roi'] else 0
        ids.append(data['symbol'].upper())
    return ids

def get_top_n_coins_symbol(n):
    page = 0
    totla_downloaded = 0
    rows = []
    while totla_downloaded < n:
        page = page + 1
        data = get_market_data(page)
        if not len(data):
            break
        totla_downloaded = totla_downloaded + len(data)
        rows.extend(data)
        time.sleep(1)
    rows = rows[0:n]
    return rows


In [47]:
# top_symbols = get_top_n_coins_symbol(top_symbols_count)

In [48]:
# top_symbols

In [49]:
def get_defi_tvl_data_df():
    defi_tvl = pd.read_sql_query("select * from notebook_defi_tvl;", __connect__())
    defi_tvl = defi_tvl.sort_values(['date', 'Symbol', 'Name'])
    defi_tvl['date'] = pd.to_datetime(defi_tvl['date'].dt.strftime('%Y-%m-%d'), format='%Y-%m-%d')
    defi_tvl = defi_tvl.groupby(['date', 'Symbol', 'Name']).tail(1)
    defi_tvl = defi_tvl.reset_index(drop=True)
    return defi_tvl
    

In [50]:
def get_supply_data_df():
    cmc_supply = pd.read_sql_query("select * from notebook_cmc_supply;", __connect__())
    cmc_supply['TVL'] = np.nan
    cgck_supply = pd.read_sql_query("select * from notebook_coingecko_supply;", __connect__())
    cgck_min_date = cgck_supply.date.min()
    cmc_supply = cmc_supply[cmc_supply['date'] < cgck_min_date]
    cmc_supply = cmc_supply[cmc_supply['Symbol'].isin(cgck_supply.Symbol.unique().tolist())]
    master_df = cmc_supply.append(cgck_supply)
    master_df['TVL Coingecko'] = master_df['TVL']
    del master_df['TVL']
    return master_df

In [51]:
def get_all_other_datasource_symbols():
    sql = 'select * from notebook_all_required_symbols_cgk;'
    symbols = pd.read_sql_query(sql, __connect__())
    return symbols

In [52]:
def get_joins_data_df(all_datasource_symbols):
    master_symbols = pd.read_sql_query("""select cgk.symbol as Symbol, cgk.`name` as Name, group_concat(cgk.tags, ',') as Tags from cgk_coin_metainfo cgk
group by cgk.symbol, cgk.name""", __connect__())
    cmc_supply = get_supply_data_df()
    db_tvl = pd.read_sql_query("select * from notebook_debank_data;", __connect__())
    defi_tvl = get_defi_tvl_data_df()
    dune_tua = pd.read_sql_query("select * from notebook_dune_tua;", __connect__())
    tt_metrics = pd.read_sql_query("select * from notebook_tt_metric_v2;", __connect__())
    defistation_tvl = pd.read_sql_query("select * from notebook_defisation_tvl;", __connect__())
    solana_tvl = pd.read_sql_query("select * from notebook_solana_tvl;", __connect__())
    master_df = pd.merge(cmc_supply, db_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, defi_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, dune_tua, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, tt_metrics, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, defistation_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, solana_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, master_symbols, left_on=['Symbol', 'Name'], right_on=['Symbol', 'Name'])
    master_df['required'] = (master_df['is_defistation'] == 1) | (master_df['is_solana'] == 1) | (master_df['is_debank'] == 1) | (master_df['is_tt_coin'] == 1) | (master_df['is_defi_tvl'] == 1) | (master_df['is_dune_tua'] == 1)
    master_df['required'] = (master_df['Name'].isin(all_datasource_symbols['name_in_coingecko']))
    del master_df['is_debank']
    del master_df['is_tt_coin']
    del master_df['is_defi_tvl']
    del master_df['is_dune_tua']
    del master_df['is_cmc_supply']
    del master_df['is_defistation']
    del master_df['is_solana']
    return master_df

In [53]:
def get_data_df(all_datasource_symbols):
    df = get_joins_data_df(all_datasource_symbols)
    df = df[(df['Symbol'].isin(all_datasource_symbols)) | (df['required'])]
    df = df.reset_index(drop=True)
    return df

In [54]:
all_data_source_symbols = get_all_other_datasource_symbols()
df = get_data_df(all_data_source_symbols)

In [55]:
len(all_data_source_symbols)

393

In [56]:
# df.columns

In [57]:
df

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Revenue TT,Revenue Supply Side TT,Revenue Protocol TT,Price/Sale,GMV TT,tvl_defistation,users_defistation,tvl_solana,Tags,required
0,2020-12-21,ADA,Cardano,0.152656,4.749512e+09,1.139902e+09,3.111248e+10,4.500000e+10,4.500000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Smart Contract Platform,",True
1,2020-12-22,ADA,Cardano,0.156179,4.859123e+09,1.138401e+09,3.111248e+10,4.500000e+10,4.500000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Smart Contract Platform,",True
2,2020-12-23,ADA,Cardano,0.136394,4.243561e+09,1.484454e+09,3.111248e+10,4.500000e+10,4.500000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Smart Contract Platform,",True
3,2020-12-24,ADA,Cardano,0.152883,4.756585e+09,1.470292e+09,3.111248e+10,4.500000e+10,4.500000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Smart Contract Platform,",True
4,2020-12-25,ADA,Cardano,0.157564,4.902201e+09,1.119918e+09,3.111248e+10,4.500000e+10,4.500000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Smart Contract Platform,",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38842,2021-07-17,SYFL,YFLink Synthetic,0.003415,0.000000e+00,6.886000e+01,0.000000e+00,NaN,5.000000e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Decentralized Finance (DeFi),",True
38843,2021-07-17,UNI,UNIVERSE Token,0.000268,0.000000e+00,0.000000e+00,0.000000e+00,NaN,9.517714e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",",True
38844,2021-07-17,WAD,Warden,0.537049,0.000000e+00,4.977000e+04,0.000000e+00,NaN,4.032103e+07,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",",True
38845,2021-07-17,XDAI,xDAI,1.000000,0.000000e+00,7.410500e+05,0.000000e+00,NaN,NaN,107066229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"xDAI Ecosystem,Cryptocurrency,Decentralized Fi...",True


In [58]:
original_df = df.copy()

In [59]:
len(original_df.Symbol.unique().tolist())

348

In [60]:
len(df.Symbol.unique().tolist())

348

In [61]:
# df.columns

In [62]:
ffill_columns = ['Users life-to-date duneanlytics', 'tvl_defistation', 'tvl_solana', 'tvl_defistation', 'Revenue TT', 'Revenue Supply Side TT', 'Revenue Protocol TT', 'Price/Sale', 'GMV TT']

def ffill_columns_values(df, columns):
    df = df.reset_index(drop=True)
    df.index = pd.DatetimeIndex(df['date'])
    for column in columns:
        df[column] = df.groupby(['Symbol','Name'])[column].ffill()
    df = df.reset_index(drop=True)
    return df

In [63]:
df = ffill_columns_values(df, ffill_columns)

In [64]:
df[df['Symbol'] == 'RAY']['tvl_solana']

32302             NaN
32303             NaN
32304             NaN
32305             NaN
32306             NaN
             ...     
32441    6.054952e+08
32442    5.628349e+08
32443    5.723276e+08
32444    5.604767e+08
32445    5.429483e+08
Name: tvl_solana, Length: 144, dtype: float64

In [65]:
df['tvl_solana'].unique()

array([           nan, 0.00000000e+00, 1.15945703e+10, 1.17926242e+10,
       1.12003160e+10, 1.12474582e+10, 1.06675689e+10, 1.08506959e+10,
       9.53953046e+09, 1.03909086e+10, 9.79477330e+09, 9.77647954e+09,
       4.45861062e+08, 4.45076346e+08, 4.45519149e+08, 4.44905498e+08,
       4.45356058e+08, 4.45759883e+08, 6.33835211e+08, 3.71837282e+08,
       3.83344711e+08, 3.30601584e+08, 2.82004268e+08, 7.19633295e+08,
       1.63888194e+08, 2.71702243e+08, 2.75747185e+08, 2.58113176e+08,
       3.24466830e+08, 1.44188980e+08, 3.25057947e+08, 2.92477941e+08,
       2.98318359e+08, 3.06566777e+08, 1.61864852e+08, 1.91964406e+08,
       1.92236865e+08, 2.02809446e+08, 1.94045869e+08, 2.16160534e+08,
       1.87777029e+08, 1.88175260e+08, 1.96680932e+08, 1.79934544e+08,
       1.63677461e+08, 1.94996265e+08, 1.97719567e+08, 2.06354244e+08,
       1.99299567e+08, 2.08085660e+08, 1.82211200e+08, 1.86853120e+08,
       1.65520729e+08, 1.64622452e+08, 1.61541029e+08, 1.62630444e+08,
      

In [66]:
df.tvl_defistation.unique()

array([      nan, 46246102., 48802600., ...,  1544130.,  1534932.,
        1748459.])

In [67]:
required_tags

['Binance Smart Chain Ecosystem',
 'Polkadot Ecosystem',
 'Cosmos Ecosystem',
 'Solana Ecosystem',
 'Avalanche Ecosystem',
 'HECO Chain Ecosystem',
 'Layer 1',
 'Decentralized Finance (DeFi)',
 'Yield Farming',
 'Decentralized Exchange Token (DEX)',
 'Governance',
 'DaoMaker Ecosystem',
 'Gaming',
 'Stablecoins',
 'Asset Manager',
 'Centralized Exchange Token (CEX)',
 'Eth 2.0 Staking',
 'Yield Aggregator',
 'Lending/Borrowing',
 'Options',
 'Launchpad']

### Adding cmc tags as column

In [68]:
def append_tags_in_columns(df):
    required_tags.sort()
    tag_df = pd.DataFrame(df['Tags'].copy())

    required_tags_map = {}
    i = 0
    for tag in required_tags:
        required_tags_map[tag] = i
        i = i + 1
        
    all_symbol_tags = []
    for index, row in tag_df.iterrows():
        tag = [''] * (len(required_tags))
        tags = row['Tags']
        tags = tags.split(',') if tags else []
        for t in tags:
            index = required_tags_map[t] if t in required_tags_map else -1
            if index > -1:
                tag[index] = 1
        all_symbol_tags.append(tag)

    tag_columns = []
    tag_columns.extend(required_tags)
    tag_column_df = pd.DataFrame(all_symbol_tags, columns = tag_columns)
    df = df.join(tag_column_df)
    del df['Tags']
    df.reset_index(inplace=True, drop=True)
    return df

In [69]:
df = append_tags_in_columns(df)

Filter symbols.

In [70]:
exceptional_coins = ['BTC', 'ETH', 'BNB', 'SOL']
df = df[(df['Symbol'].isin(exceptional_coins)) | (df['Decentralized Finance (DeFi)'] == 1) | (df['required'])]
del df['required']
# df = df[(df['defi'] == 1)]



In [71]:
len(df.Name.unique())

356

In [72]:
original_df[original_df['Symbol'] == 'MATIC']

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Revenue TT,Revenue Supply Side TT,Revenue Protocol TT,Price/Sale,GMV TT,tvl_defistation,users_defistation,tvl_solana,Tags,required
3914,2020-12-21,MATIC,Polygon,0.018155,8.711707e+07,1.045840e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,0.218400,0.218400,0.0,1.945808e+06,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
3915,2020-12-22,MATIC,Polygon,0.018646,8.947142e+07,8.713462e+06,4.798458e+09,NaN,1.000000e+10,NaN,...,0.233426,0.233426,0.0,1.818049e+06,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
3916,2020-12-23,MATIC,Polygon,0.015204,7.295766e+07,1.394960e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,0.204871,0.204871,0.0,1.836513e+06,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
3917,2020-12-24,MATIC,Polygon,0.016463,7.899817e+07,1.046229e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,0.226584,0.226584,0.0,1.488616e+06,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
3918,2020-12-25,MATIC,Polygon,0.016621,7.975443e+07,1.062611e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,0.167761,0.167761,0.0,1.592512e+06,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4115,2021-07-12,MATIC,Polygon,1.040000,6.597964e+09,2.782265e+08,6.330555e+09,1.000000e+10,1.000000e+10,7.859113e+09,...,18875.305722,18875.305722,0.0,2.047577e+03,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
4116,2021-07-14,MATIC,Polygon,0.893388,5.645997e+09,4.998300e+08,6.330555e+09,1.000000e+10,1.000000e+10,8.278510e+09,...,19232.433975,19232.433975,0.0,1.655053e+03,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
4117,2021-07-15,MATIC,Polygon,0.909374,5.754947e+09,4.736629e+08,6.330555e+09,1.000000e+10,1.000000e+10,8.340342e+09,...,19807.168638,19807.168638,0.0,1.582170e+03,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True
4118,2021-07-16,MATIC,Polygon,0.861658,5.457486e+09,3.746123e+08,6.330555e+09,1.000000e+10,1.000000e+10,7.641654e+09,...,16291.214370,16291.214370,0.0,1.462994e+03,NaN,NaN,NaN,NaN,"Polygon Ecosystem,",True


In [73]:
df[df['Symbol'] == 'MATIC']

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,HECO Chain Ecosystem,Launchpad,Layer 1,Lending/Borrowing,Options,Polkadot Ecosystem,Solana Ecosystem,Stablecoins,Yield Aggregator,Yield Farming
3914,2020-12-21,MATIC,Polygon,0.018155,8.711707e+07,1.045840e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,,,,,,,,,,
3915,2020-12-22,MATIC,Polygon,0.018646,8.947142e+07,8.713462e+06,4.798458e+09,NaN,1.000000e+10,NaN,...,,,,,,,,,,
3916,2020-12-23,MATIC,Polygon,0.015204,7.295766e+07,1.394960e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,,,,,,,,,,
3917,2020-12-24,MATIC,Polygon,0.016463,7.899817e+07,1.046229e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,,,,,,,,,,
3918,2020-12-25,MATIC,Polygon,0.016621,7.975443e+07,1.062611e+07,4.798458e+09,NaN,1.000000e+10,NaN,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4115,2021-07-12,MATIC,Polygon,1.040000,6.597964e+09,2.782265e+08,6.330555e+09,1.000000e+10,1.000000e+10,7.859113e+09,...,,,,,,,,,,
4116,2021-07-14,MATIC,Polygon,0.893388,5.645997e+09,4.998300e+08,6.330555e+09,1.000000e+10,1.000000e+10,8.278510e+09,...,,,,,,,,,,
4117,2021-07-15,MATIC,Polygon,0.909374,5.754947e+09,4.736629e+08,6.330555e+09,1.000000e+10,1.000000e+10,8.340342e+09,...,,,,,,,,,,
4118,2021-07-16,MATIC,Polygon,0.861658,5.457486e+09,3.746123e+08,6.330555e+09,1.000000e+10,1.000000e+10,7.641654e+09,...,,,,,,,,,,


### adding columns like TVL_max, Defi Volume_max, users_life_max and etc.

In [74]:
def add_other_columns(df):
    # df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse']].max(axis=1)
    # df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse', 'tvl_defistation']].max(axis=1)
    df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse', 'tvl_defistation', 'tvl_solana', 'TVL Coingecko']].max(axis=1)
    df['GMV'] = df['GMV TT']
    df['Defi Volume DeBank'] = (df['Trading Volume Debank'].fillna(0) + df['Borrowing Volume Debank'].fillna(0)).replace({'0': np.nan, 0: np.nan})
    # add defi volume max as max of gmv from tokenterminal and defi volume debank
    df['Defi Volume_max'] = df[['GMV', 'Defi Volume DeBank']].max(axis=1)
    # add defi volume max as max of gmv from tokenterminal and defi volume debank
    df['Defi Volume_max'] = df[['GMV', 'Defi Volume DeBank']].max(axis=1)
    df['max_test'] = df[['MAX Supply CMC', 'Total Supply CMC']].max(axis=1)
    df['fully_diluted_mkt_cap'] = df.max_test * df.Price
    df["total_revenue"] = df['Revenue TT']
    df["total_supply_revenue"] = df['Revenue Supply Side TT']
    df["total_protocol_revenue"] = df['Revenue Protocol TT']
    df['users_life_max'] = df[['users_defistation', 'Users life-to-date duneanlytics']].max(axis=1)
    del df['max_test']
    df = df.sort_values('date')
    return df

In [75]:
df = add_other_columns(df)

In [76]:
def select_required_metrics_coins_only(df):

    def fillter_coins_basis_metrics(df, column, symbol):
        tmp_df = df[df['Symbol'] == symbol][column].isnull()
        return tmp_df.sum() == tmp_df.count()

    to_delete_symbols = []
    required_metrics = ['TVL_max', 'Defi Volume_max', 'total_revenue']
    symbols = df.Symbol.unique().tolist()
    for symbol in symbols:
        to_delete = True
        for metric in required_metrics:
            to_delete = fillter_coins_basis_metrics(df, metric, symbol)
            if not to_delete:
                break
        to_delete_symbols.append([symbol, to_delete])
    to_delete_symbols_df = pd.DataFrame(to_delete_symbols, columns=['Symbol', 'to_delete'])
    to_delete_symbols_df = to_delete_symbols_df[to_delete_symbols_df['to_delete']]
    to_delete_symbols = to_delete_symbols_df['Symbol'].values.tolist()
    df2 = df.drop(df[df['Symbol'].isin(to_delete_symbols)].index)
    df2 = df2.reset_index(drop=True)
    return df2

In [77]:
df = select_required_metrics_coins_only(df)
df

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Yield Farming,TVL_max,GMV,Defi Volume DeBank,Defi Volume_max,fully_diluted_mkt_cap,total_revenue,total_supply_revenue,total_protocol_revenue,users_life_max
0,2020-12-21,RARI,Rarible,2.605508,2228569.97,1227740.42,8.553304e+05,NaN,2.500000e+07,NaN,...,,NaN,4.859901e+04,NaN,4.859901e+04,6.513769e+07,2429.950456,0.0,2429.950456,NaN
1,2020-12-21,KAVA,Kava.io,1.493707,70019348.83,23278351.50,4.687623e+07,NaN,1.135568e+08,NaN,...,,NaN,NaN,NaN,NaN,1.696206e+08,NaN,NaN,NaN,NaN
2,2020-12-21,RAMP,RAMP,0.038369,4600196.38,300333.82,1.198932e+08,NaN,1.000000e+09,NaN,...,,2.347403e+06,NaN,NaN,NaN,3.836912e+07,NaN,NaN,NaN,NaN
3,2020-12-21,PRQ,PARSIQ,0.408170,42607025.18,4924221.66,1.043856e+08,5.000000e+08,5.000000e+08,NaN,...,,NaN,NaN,NaN,NaN,2.040848e+08,NaN,NaN,NaN,NaN
4,2020-12-21,PERP,Perpetual Protocol,1.061385,16946247.63,746906.05,1.596617e+07,NaN,1.500000e+08,NaN,...,,NaN,4.245194e+06,NaN,4.245194e+06,1.592077e+08,4237.944176,0.0,4237.944176,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32193,2021-07-17,POWER,UniPower,0.537834,536893.00,1153.92,9.902500e+05,NaN,1.000000e+06,NaN,...,,3.926038e+05,NaN,NaN,NaN,5.378340e+05,NaN,NaN,NaN,NaN
32194,2021-07-17,PIPT,Power Index Pool Token,2.700000,2835643.00,32078.00,1.040844e+06,1.040844e+06,1.040844e+06,16552711.0,...,1,1.655271e+07,NaN,NaN,NaN,2.810279e+06,NaN,NaN,NaN,NaN
32195,2021-07-17,FLUX,Flux,0.089821,16080492.00,115300.00,1.816186e+08,NaN,4.400000e+08,NaN,...,,8.227704e+06,NaN,NaN,NaN,3.952124e+07,NaN,NaN,NaN,NaN
32196,2021-07-17,STBZ,Stabilize,0.878677,591323.00,35146.00,6.656176e+05,NaN,7.518953e+05,NaN,...,1,1.004264e+06,NaN,NaN,NaN,6.606731e+05,NaN,NaN,NaN,NaN


In [78]:
cmc_symbol_name_df = get_all_other_datasource_symbols()
cmc_symbol_name_df['Symbol'] = cmc_symbol_name_df['symbol']
cmc_symbol_name_df['Name'] = cmc_symbol_name_df['name_in_coingecko']

In [79]:
cmc_symbol_name_df

,symbol,name_in_coingecko,Symbol,Name
0,$KING,King Swap,$KING,King Swap
1,1INCH,1inch,1INCH,1inch
2,AAVE,Aave,AAVE,Aave
3,ACS,Acryptos,ACS,Acryptos
4,ADD,Add.xyz (NEW),ADD,Add.xyz (NEW)
...,...,...,...,...
388,SOL,Solana,SOL,Solana
389,STEP,Step Finance,STEP,Step Finance
390,USDC,USD Coin,USDC,USD Coin
391,USDT,Tether,USDT,Tether


### Adding all ratios

In [80]:
# add n day users
def calculate_nDay_users(df, days):
    column_name = f"total_users_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['24hr_Debank_users_max'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]

In [81]:
def calculate_nDay_revenue(df, days):
    column_name = f"total_revenue_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['total_revenue'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [82]:
# function for defi volume sums
# substitute for volume cmc sum
# changing defi volume debank to Defi Volume_max
def calculate_nDay_defi_volume(df, days):
    column_name = f"total_defi_volume_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['Defi Volume_max'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
            

In [83]:
def calculate_nDay_change(df, days, column, column_name):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].pct_change(days)
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    
def calculate_nDay_std(df, days, column, column_name, div_by_column='Price'):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].rolling(days).std() / temp_df[div_by_column]
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [84]:
def add_ratios_and_pct_change(df):
    calculate_nDay_revenue(df, 7)
    calculate_nDay_revenue(df, 30)
    #calculate_nDay_trading_volume(df, 30)
    calculate_nDay_defi_volume(df, 30)

    # add n day users
    calculate_nDay_users(df, 30)

    # add 1, 3, 7, 30 day defi volume pct change here
    # added column calculations
    #
    calculate_nDay_change(df, 1, 'Defi Volume_max', 'Defi_volume_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Defi Volume_max', 'Defi_volume_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Defi Volume_max', 'Defi_volume_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Defi Volume_max', 'Defi_volume_pct_chg_30d')
    #
    # flip such that mc in numerator 
    #df['VOL : MC'] = df['Volume CMC'] / df['Marketcap CMC']
    df['MC : VOL'] = df['Marketcap CMC'] / df['Volume CMC'] 
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : VOL', 'MC_VOL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : VOL', 'MC_VOL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : VOL', 'MC_VOL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : VOL', 'MC_VOL_pct_chg_30d')
    #

    # flip such that mc in numerator
    #df['VOL : DMC'] = df['Volume CMC'] / df['fully_diluted_mkt_cap']
    df['DMC : VOL'] = df['fully_diluted_mkt_cap'] / df['Volume CMC']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : VOL', 'DMC_VOL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : VOL', 'DMC_VOL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : VOL', 'DMC_VOL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : VOL', 'DMC_VOL_pct_chg_30d')
    #



    df['MC : TVL'] = df['Marketcap CMC'] / df['TVL_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : TVL', 'MC_TVL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : TVL', 'MC_TVL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : TVL', 'MC_TVL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : TVL', 'MC_TVL_pct_chg_30d')
    #


    df['DMC : TVL'] = df['fully_diluted_mkt_cap'] / df['TVL_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : TVL', 'DMC_TVL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : TVL', 'DMC_TVL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : TVL', 'DMC_TVL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : TVL', 'DMC_TVL_pct_chg_30d')
    #


    df['MC : users (1d)'] = df['Marketcap CMC'] / df['24hr_Debank_users_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : users (1d)', 'MC_users_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : users (1d)', 'MC_users_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : users (1d)', 'MC_users_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : users (1d)', 'MC_users_pct_chg_30d')
    #




    df['DMC : users (1d)'] = df['fully_diluted_mkt_cap'] / df['24hr_Debank_users_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : users (1d)', 'DMC_users_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : users (1d)', 'DMC_users_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : users (1d)', 'DMC_users_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : users (1d)', 'DMC_users_pct_chg_30d')
    #



    df['MC : transactions (1d)'] = df['Marketcap CMC'] / df['Transactions Count Debank']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : transactions (1d)', 'MC_transactions_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : transactions (1d)', 'MC_transactions_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : transactions (1d)', 'MC_transactions_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : transactions (1d)', 'MC_transactions_pct_chg_30d')
    #

    df['DMC : transactions (1d)'] = df['fully_diluted_mkt_cap'] / df['Transactions Count Debank']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_30d')
    #

    #df['MC : DEX Vol (1d)'] = df['Marketcap CMC'] / df['Trading Volume Debank']
    # add
    df['MC : Defi Vol (1d)'] = df['Marketcap CMC'] / df['Defi Volume_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_30d')
    #


    df['MC : Defi Vol (30d)'] = df['Marketcap CMC'] / df['total_defi_volume_30d']
    #df['DMC : DEX Vol (1d)'] = df['fully_diluted_mkt_cap'] / df['Trading Volume Debank']
    #df['DMC : DEX Vol (30d)'] = df['fully_diluted_mkt_cap'] / df['total_volume_30d']



    df['MC : revenue'] = df['Marketcap CMC'] / df['total_revenue']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : revenue', 'MC_revenue_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : revenue', 'MC_revenue_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : revenue', 'MC_revenue_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : revenue', 'MC_revenue_pct_chg_30d')
    #

    df['DMC : revenue'] = df['fully_diluted_mkt_cap'] / df['total_revenue']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : revenue', 'DMC_revenue_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : revenue', 'DMC_revenue_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : revenue', 'DMC_revenue_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : revenue', 'DMC_revenue_pct_chg_30d')
    #


    df['MC : total_users_30d'] = df['Marketcap CMC'] / df['total_users_30d']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_30d')
    #

    df['DMC : total_users_30d'] = df['fully_diluted_mkt_cap'] / df['total_users_30d']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_30d')
    #

    df['MC : Users_life'] = df['Marketcap CMC'] / df['users_life_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : Users_life', 'MC_Users_life_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : Users_life', 'MC_Users_life_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : Users_life', 'MC_Users_life_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : Users_life', 'MC_Users_life_pct_chg_30d')
    #
    df['DMC : Users_life'] = df['fully_diluted_mkt_cap'] / df['users_life_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : Users_life', 'DMC_Users_life_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : Users_life', 'DMC_Users_life_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : Users_life', 'DMC_Users_life_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : Users_life', 'DMC_Users_life_pct_chg_30d')
    #
    calculate_nDay_change(df, 1, 'Price', 'Price_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Price', 'Price_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Price', 'Price_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Price', 'Price_pct_chg_30d')
    calculate_nDay_std(df, 7, 'Price', 'Price_7day_volatility_pct', div_by_column='Price')
    calculate_nDay_std(df, 30, 'Price', 'Price_30day_volatility_pct', div_by_column='Price')
    calculate_nDay_std(df, 90, 'Price', 'Price_90day_volatility_pct', div_by_column='Price')
    #
    calculate_nDay_change(df, 1, 'Marketcap CMC', 'MC_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Marketcap CMC', 'MC_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Marketcap CMC', 'MC_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Marketcap CMC', 'MC_pct_chg_30d')
    calculate_nDay_std(df, 30, 'Marketcap CMC', 'MC_30day_volatility_pct', div_by_column='Marketcap CMC')
    calculate_nDay_std(df, 90, 'Marketcap CMC', 'MC_90day_volatility_pct', div_by_column='Marketcap CMC')
    #
    calculate_nDay_change(df, 1, 'fully_diluted_mkt_cap', 'DMC_pct_chg_1d')
    calculate_nDay_change(df, 3, 'fully_diluted_mkt_cap', 'DMC_pct_chg_3d')
    calculate_nDay_change(df, 7, 'fully_diluted_mkt_cap', 'DMC_pct_chg_7d')
    calculate_nDay_change(df, 30, 'fully_diluted_mkt_cap', 'DMC_pct_chg_30d')
    calculate_nDay_std(df, 30, 'fully_diluted_mkt_cap', 'DMC_30day_volatility_pct', div_by_column='fully_diluted_mkt_cap')
    calculate_nDay_std(df, 90, 'fully_diluted_mkt_cap', 'DMC_90day_volatility_pct', div_by_column='fully_diluted_mkt_cap')
    #
    calculate_nDay_change(df, 1, 'Circulating supply CMC', 'CS_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Circulating supply CMC', 'CS_pct_chg_d')
    calculate_nDay_change(df, 7, 'Circulating supply CMC', 'CS_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Circulating supply CMC', 'CS_pct_chg_30d')
    calculate_nDay_std(df, 30, 'Circulating supply CMC', 'CS_30day_volatility_pct', div_by_column='Circulating supply CMC')
    calculate_nDay_std(df, 90, 'Circulating supply CMC', 'CS_90day_volatility_pct', div_by_column='Circulating supply CMC')
    #
    calculate_nDay_change(df, 1, 'Volume CMC', 'Volume_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Volume CMC', 'Volume_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Volume CMC', 'Volume_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Volume CMC', 'Volume_pct_chg_30d')
    calculate_nDay_std(df, 30, 'Volume CMC', 'Volume_30day_volatility_pct', div_by_column='Volume CMC')
    calculate_nDay_std(df, 90, 'Volume CMC', 'Volume_90day_volatility_pct', div_by_column='Volume CMC')
    #
    calculate_nDay_change(df, 1, 'TVL_max', 'TVL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'TVL_max', 'TVL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'TVL_max', 'TVL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'TVL_max', 'TVL_pct_chg_30d')
    calculate_nDay_std(df, 30, 'TVL_max', 'TVL_30day_volatility_pct', div_by_column='TVL_max')
    calculate_nDay_std(df, 90, 'TVL_max', 'TVL_90day_volatility_pct', div_by_column='TVL_max')
    #
    calculate_nDay_change(df, 1, 'total_revenue', 'Rev_pct_chg_1d')
    calculate_nDay_change(df, 3, 'total_revenue', 'Rev_pct_chg_3d')
    calculate_nDay_change(df, 7, 'total_revenue', 'Rev_pct_chg_7d')
    calculate_nDay_change(df, 30, 'total_revenue', 'Rev_pct_chg_30d')
    
    calculate_nDay_change(df, 1, 'Circulating supply CMC', 'emission_rate')
    
    calculate_nDay_std(df, 30, 'total_revenue', 'Rev_30day_volatility_pct', div_by_column='total_revenue')
    calculate_nDay_std(df, 90, 'total_revenue', 'Rev_90day_volatility_pct', div_by_column='total_revenue')
    df['Annual revenue (estimate)'] = df['total_revenue_30d'] * (1 + df['Rev_pct_chg_30d'])*12
    return df


In [85]:
# df = add_ratios_and_pct_change(df)

In [86]:
df

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Yield Farming,TVL_max,GMV,Defi Volume DeBank,Defi Volume_max,fully_diluted_mkt_cap,total_revenue,total_supply_revenue,total_protocol_revenue,users_life_max
0,2020-12-21,RARI,Rarible,2.605508,2228569.97,1227740.42,8.553304e+05,NaN,2.500000e+07,NaN,...,,NaN,4.859901e+04,NaN,4.859901e+04,6.513769e+07,2429.950456,0.0,2429.950456,NaN
1,2020-12-21,KAVA,Kava.io,1.493707,70019348.83,23278351.50,4.687623e+07,NaN,1.135568e+08,NaN,...,,NaN,NaN,NaN,NaN,1.696206e+08,NaN,NaN,NaN,NaN
2,2020-12-21,RAMP,RAMP,0.038369,4600196.38,300333.82,1.198932e+08,NaN,1.000000e+09,NaN,...,,2.347403e+06,NaN,NaN,NaN,3.836912e+07,NaN,NaN,NaN,NaN
3,2020-12-21,PRQ,PARSIQ,0.408170,42607025.18,4924221.66,1.043856e+08,5.000000e+08,5.000000e+08,NaN,...,,NaN,NaN,NaN,NaN,2.040848e+08,NaN,NaN,NaN,NaN
4,2020-12-21,PERP,Perpetual Protocol,1.061385,16946247.63,746906.05,1.596617e+07,NaN,1.500000e+08,NaN,...,,NaN,4.245194e+06,NaN,4.245194e+06,1.592077e+08,4237.944176,0.0,4237.944176,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32193,2021-07-17,POWER,UniPower,0.537834,536893.00,1153.92,9.902500e+05,NaN,1.000000e+06,NaN,...,,3.926038e+05,NaN,NaN,NaN,5.378340e+05,NaN,NaN,NaN,NaN
32194,2021-07-17,PIPT,Power Index Pool Token,2.700000,2835643.00,32078.00,1.040844e+06,1.040844e+06,1.040844e+06,16552711.0,...,1,1.655271e+07,NaN,NaN,NaN,2.810279e+06,NaN,NaN,NaN,NaN
32195,2021-07-17,FLUX,Flux,0.089821,16080492.00,115300.00,1.816186e+08,NaN,4.400000e+08,NaN,...,,8.227704e+06,NaN,NaN,NaN,3.952124e+07,NaN,NaN,NaN,NaN
32196,2021-07-17,STBZ,Stabilize,0.878677,591323.00,35146.00,6.656176e+05,NaN,7.518953e+05,NaN,...,1,1.004264e+06,NaN,NaN,NaN,6.606731e+05,NaN,NaN,NaN,NaN


In [87]:
def arrange_columns(df):
    columns = df.columns.tolist()
    for tag in required_tags:
        columns.remove(tag)
    columns.extend(required_tags)
    return df[columns]

In [88]:
df = arrange_columns(df)

### Generating neg div score

In [89]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import numpy as np
import pymysql
import traceback2


def read_csv_as_df():
    df = pd.read_csv('/tmp/coins_data.csv')
    df.index = pd.DatetimeIndex(df['time'])
    del df['time']
    df = df[df.index >= datetime(2020, 12, 26)]
    return df

def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_defi_neg_div_score(df, date):
    connection = __connect__()
    cursor = get_cursor(connection)
    dt = date.timestamp()
    for i, row in df.iterrows():
        try:
            sys_name = row["Symbol"]
            name = sys_name.split("###")[1]
            symbol = sys_name.split("###")[0]
            print(dt, date, symbol, name, row['C_Price'], row['3_Day_LB'], row['5_Day_LB'])
            sql = "INSERT INTO `defi_neg_div_score` (`timestamp_dt`, `symbol`, `name`, `price`, `3_Day_LB`, `5_Day_LB`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, symbol, name, row['C_Price'], row['3_Day_LB'], row['5_Day_LB']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

def clean_data(df):
    index = df.index.values
#     print(index)
    if 'USDT' in index:
        del df['USDT']
    if 'USDC' in index:
        del df['USDC']
    if 'DAI' in index:
        del df['DAI']
    if 'BUSD' in index:
        del df['BUSD']
    if 'HUSD' in index:
        del df['HUSD']
    if 'TUSD' in index:
        del df['TUSD']
    if 'BTC' in index:
        del df['BTC']
    data_values_df = pd.DataFrame(df.isna().sum(), columns=['nan_count'])
    tmp_df = pd.DataFrame(df.count(), columns=['values_count'])
    data_values_df['values_count'] = tmp_df['values_count']
    data_values_df['rows_count'] = data_values_df['values_count'] + data_values_df['nan_count']
#     print(data_values_df)
    delete_coin_df = data_values_df[data_values_df['nan_count'] > (data_values_df['values_count'] * .20)]
    delete_coin_df.sort_values(['nan_count'])
    symbols_del = delete_coin_df.index.tolist()
    for symbol in symbols_del:
        print(f'Deleteing Coin: {symbol}')
        del df[symbol]
    df.ffill(inplace=True)
    df = df.dropna(axis=1)
    df.index = df.index
    return df


def get_todays_portfolio_balance(trades):
    balance = 0
    for symbol in trades:
        balance = balance + trades[symbol][3]
    return balance


def make_entry_symbol(df, trading_date, symbol, last_trade, fund):
    last_trade = last_trade.copy()
    todays_close = get_close_price_on(df, symbol, trading_date)
    if not len(last_trade):
        last_trade = [trading_date, todays_close, fund / todays_close, fund, 0]
        return last_trade
    last_trade[0] = trading_date
    last_trade[1] = todays_close
    last_trade[3] = last_trade[2] * todays_close
    last_trade[4] = last_trade[4] + 1
#     print(last_trade)
    return last_trade


def make_exit_symbol(df, trading_date, symbol, last_trade):
    quantity = last_trade[2]
    todays_close = get_close_price_on(df, symbol, trading_date)
    return quantity * todays_close


def get_not_in_list(new_list, old_list):
    not_in_new = []
    for symbol in old_list:
        if symbol not in new_list:
            not_in_new.append(symbol)
    return not_in_new


def get_commons(symbols1, symbols2):
    symbols_list = []
    for symbol in symbols2:
        if symbol in symbols1:
            symbols_list.append(symbol)
    return symbols_list


def get_joins(symbols1, symbols2):
    symbols_list = symbols1.copy()
    for symbol in symbols2:
        if symbol not in symbols1:
            symbols_list.append(symbol)
    return symbols_list


def get_todays_top_n_coin(df, trade_date, top_n, order='tail'):
    data = df.loc[trade_date]
    data = pd.DataFrame(data)
    data = data.rename(columns={data.columns[0]: 'C'})
    todays_close_df = pd.DataFrame(data)
    if order == 'head':
        todays_close_df = todays_close_df.sort_values(['C']).head(top_n)
    else:
        todays_close_df = todays_close_df.sort_values(['C']).tail(top_n)
    return todays_close_df.index.values.tolist()


def get_next_trading_day_as_index(start_date, hour):
    return pd.to_datetime((start_date + timedelta(days=1)).replace(hour=hour, minute=0, second=0).strftime("%Y %m %d %H:%M:00"))


def get_close_price_on(df, symbol, trade_date):
    data = df.loc[trade_date]
    data = pd.DataFrame(data)
    data = data.rename(columns={data.columns[0]: 'C'})
    todays_close_df = pd.DataFrame(data)
    try:
        return todays_close_df.loc[symbol]['C']
    except Exception:
        return 0


def get_qualified_coins(trade_date, factors_df, factors_count):
    factors_symbols = []
    index = 0
    for factor_df in factors_df:
        s1 = get_todays_top_n_coin(factor_df, trade_date, factors_count[index])
        index = index + 1
        factors_symbols.append(s1)
    symbols = factors_symbols[0]
    for index in range(1, len(factors_symbols)):
        symbols = get_commons(symbols, factors_symbols[index])
    return symbols


def update_coins_performance(coins_performence, todays_trading_symbols):
    symbol_performance = [0, 0]
    for symbol in todays_trading_symbols:
        if symbol in coins_performence:
            symbol_performance = coins_performence[symbol]
        else:
            symbol_performance = [0, 0]
            coins_performence[symbol] = symbol_performance
        symbol_performance[0] = symbol_performance[0] + 1


def update_total_days_count_in_coin_performance(coins_performence, days_count):
    for symbol in coins_performence:
        coins_performence[symbol][1] = days_count


def plot_portfolio_performence(trade_date, df, factors_df, factors_count):
    trades = {}
    trades['c-f'] = [trade_date, 0, 0, 0]
    last_day_trading_symbols = []
    is_first_day_trading = True
    portfolio_performence = []
    end_time = df.index.values.max()
    end_time = (end_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    coins_performence = {}
    days_count = 0
    while trade_date.timestamp() < end_time:
        days_count = days_count + 1
        todays_trading_symbols = get_qualified_coins(trade_date, factors_df, factors_count)
        update_coins_performance(coins_performence, todays_trading_symbols)
        if is_first_day_trading:
            is_first_day_trading = False
            if not len(todays_trading_symbols):
                trades['c-f'][3] = 100000
                continue
            fund = 100000 / len(todays_trading_symbols)
            for symbol in todays_trading_symbols:
                trade = make_entry_symbol(df, trade_date, symbol, [], fund)
                trades[symbol] = trade
#             is_first_day_trading = False
            last_day_trading_symbols = todays_trading_symbols.copy()
            continue
        tmp_trades = trades

        trades = {}
        exit_fund = tmp_trades['c-f'][3]
        todays_exit_symbols = get_not_in_list(todays_trading_symbols, last_day_trading_symbols)
        todays_common_symbols = get_commons(todays_trading_symbols, last_day_trading_symbols)
        todays_new_entry_symbols = get_not_in_list(todays_common_symbols, todays_trading_symbols)

        new_symbol = []
        for symbol_c in todays_common_symbols:
            trade = tmp_trades[symbol_c]
            if trade[4] > 5:
                exit_fund = exit_fund + make_exit_symbol(df, trade_date, symbol_c, trade)
                if symbol_c in todays_trading_symbols:
                    todays_new_entry_symbols.append(symbol_c)
                continue
            trade = make_entry_symbol(df, trade_date, symbol_c, trade, 0)
            trades[symbol_c] = trade

        for exit_symbol in todays_exit_symbols:
            exit_trade = tmp_trades[exit_symbol]
            fund = make_exit_symbol(df, trade_date, exit_symbol, exit_trade)
            exit_fund = exit_fund + fund
        if len(todays_new_entry_symbols):
            new_entry_fund = exit_fund / len(todays_new_entry_symbols)
            exit_fund = 0
            for entry_symbol in todays_new_entry_symbols:
                trade = make_entry_symbol(df, trade_date, entry_symbol, [], new_entry_fund)
                trades[entry_symbol] = trade
                new_symbol.append(entry_symbol)
        tmp_trades['c-f'][3] = exit_fund
        tmp_trades['c-f'][0] = trade_date
        trades['c-f'] = tmp_trades['c-f']
#         if tmp_trades['c-f'][3] > 0:
#             print(f"Uninvested amount as on {tmp_trades['c-f'][0]} is {tmp_trades['c-f'][3]}")
        portfolio_performence.append([get_todays_portfolio_balance(trades), trade_date])
        trade_date = get_next_trading_day_as_index(trade_date, trade_date.hour)
        last_day_trading_symbols = get_joins(todays_common_symbols, new_symbol)
    update_total_days_count_in_coin_performance(coins_performence, days_count)
    print(f'Last Trading Day: {trade_date}. Currently Coins in Portfolio: \n {todays_trading_symbols}')
    return [portfolio_performence, todays_trading_symbols, coins_performence]


def plot_portfolio_for_config(close_df, factors_pct, factors_df):
    total_symbols_count = len(close_df.columns.values.tolist())
    factors_count = []
    for factor_pct in factors_pct:
        factors_count.append(int(total_symbols_count / 100 * factor_pct))

    start_time = factors_df[0].index.values.min()
    start_time_secs = (start_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    start_time = datetime.utcfromtimestamp(start_time_secs)
    trade_date = start_time + timedelta(days=0)
    trade_date = get_next_trading_day_as_index(trade_date, trade_date.hour)
    print(f'Trading Start Date: {trade_date}')
    portfolio = plot_portfolio_performence(trade_date, close_df, factors_df, factors_count)
    portfolio_performence = portfolio[0]
    performance_df = pd.DataFrame(portfolio_performence, columns=['balance', 'time'])
    performance_df.index = pd.DatetimeIndex(performance_df['time'])
    del performance_df['time']
    performance_df.plot(figsize=[18, 10])
#     plt.savefig(f'/tmp/portfolio_{"_".join(str(x) for x in factors_pct)}.png')

    print(f'Current Portfilio Balance: {portfolio_performence[-1][0]}')
    return portfolio[1:]


def get_rsq(x):
    X = x.values.reshape(-1, 1)
    Y = np.arange(1, len(X) + 1).reshape(-1, 1)
    linear_regressor = LinearRegression(fit_intercept=True)
    linear_regressor.fit(Y, X)
    return linear_regressor.score(Y, X)
#     return r2_score(Y, X)


def get_portfolio_version_coins(close_df, factors_df):
    # factors_df = [_7_days_pct_change_df, _days_sumed_pct_change_df, _6_hour_rsq_df, _pure_div_score]
    factors = [25, 10, 10, 99]
    coins_performance = plot_portfolio_for_config(close_df, factors, factors_df)
    coin_performance_df = pd.DataFrame.from_dict(coins_performance[1], orient='index', columns=['Occurrences', 'Total Trading Day'])
    coin_performance_df = coin_performance_df.sort_values(['Occurrences'], ascending=True)
#     coin_performance_df.to_csv(f'/tmp/portfolio_{"_".join(str(x) for x in factors)}_coin_performance.csv')
    s1 = coins_performance[0]
    factors = [25, 99, 50, 7.5]
    coins_performance = plot_portfolio_for_config(close_df, factors, factors_df)
    coin_performance_df = pd.DataFrame.from_dict(coins_performance[1], orient='index', columns=['Occurrences', 'Total Trading Day'])
    coin_performance_df = coin_performance_df.sort_values(['Occurrences'], ascending=True)
#     coin_performance_df.to_csv(f'/tmp/portfolio_{"_".join(str(x) for x in factors)}_coin_performance.csv')
    s2 = coins_performance[0]
    return get_joins(s1, s2)


def run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, day):
    _7_days_pct_change_df = _6_hour_resampled_df.pct_change(periods=day * 4)
    factors_df[0] = _7_days_pct_change_df
    return get_portfolio_version_coins(close_df, factors_df)


def get_coins_price(close_df, symbols):
    df = close_df.iloc[[-1]]
    symbols_price = f"As On {df.index.values[0]}<br>"
    for symbol in symbols:
        symbols_price = f'{symbols_price} {symbol}: ${df.iloc[0][symbol]},'
    return symbols_price


def get_lookback_df(close_df, lookback_symbols):
    close_df = close_df.iloc[[-1]]
    symbols = []
    for lookback in lookback_symbols:
        symbols = get_joins(symbols, lookback_symbols[lookback])
    data = []
    for symbol in symbols:
        data.append([
            symbol,
            close_df.iloc[0][symbol],
            symbol in lookback_symbols[3],
            symbol in lookback_symbols[5]
        ])
    df = pd.DataFrame(data, columns=['Symbol', 'C_Price', '3_Day_LB', '5_Day_LB'])
    return df


def run_algo(close_df):
    _6_hour_resampled_df = close_df.resample('6H').last()
    _6_hour_pct_change = _6_hour_resampled_df.pct_change(periods=1)
    _days_sumed_pct_change_df = _6_hour_pct_change.rolling(window=4).sum()
    _pure_div_score = _6_hour_pct_change.applymap(lambda x: x if x < 0 else 0)
    _pure_div_score = _pure_div_score.rolling(window=4).sum()
    _6_hour_rsq_df = _6_hour_resampled_df.rolling(window=28).apply(get_rsq)
    factors_df = [None, _days_sumed_pct_change_df, _6_hour_rsq_df, _pure_div_score]
    # return run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, 7)
    days = [3, 5]
    msg = ""
    lookback_symbols = {}
    for day in days:
        msg = f'{msg}<br> {day}-day Lookback. <br>Coins Symbol in Portfolio are as below:<br>'
        symbols = run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, day)
        lookback_symbols[day] = symbols
        msg = f'{msg} {get_coins_price(close_df, symbols)}<br><br>'
    lookback_df = get_lookback_df(close_df, lookback_symbols)
    return lookback_df


def delete_unnessecary_columns(df):
    df = df.drop_duplicates(keep='first')
    return df


def convert_symbols_to_columns(df):
    df = df.pivot(columns='symbol', values='C')
    return df


def get_symbols_close(ids):
    connection = __connect__()
    s = ",".join([f'"{s}"' for s in ids])
    sql = f"select * from coingecko_coin_hist_data where symbol in ({s})"
    print(sql)
    df = pd.read_sql(sql, connection)
    connection.close()
    return df

def get_symbols_close_data_df(data_df):
    ids = data_df['Symbol'].unique().tolist()
    df = get_symbols_close(ids)
    df['sym-name']  = df['symbol'] +'###'+df['name']
    df['symbol'] = df['sym-name']
    df['C'] = df['price']
    df = df[['symbol', 'C', 'timestamp_dt'] ].copy()
    df['timestamp_dt'] = pd.to_datetime(df['timestamp_dt'])
    df.index = pd.DatetimeIndex(df['timestamp_dt'])
    df.index = df.index + timedelta(seconds=60)
    df = df[df.index >= datetime(2021, 1, 15)]
    df = delete_unnessecary_columns(df)
    df = convert_symbols_to_columns(df)
    return df


def run_backtest(close_df):
    df = clean_data(close_df)
    print(f'1st Trading Day: {df.index.min()}. Last Trading Day: {df.index.max()}')
    symbols_neg_div_df = run_algo(df)
    return symbols_neg_div_df

def generate_defi_neg_div_score(data_df, start_date):
    close_df = get_symbols_close_data_df(data_df)
    close_df = close_df[close_df.index < start_date].copy()
    symbols_neg_div_df = run_backtest(close_df)
    date = datetime(start_date.year, start_date.month, start_date.day)
    insert_defi_neg_div_score(symbols_neg_div_df, date)
    
    


In [90]:
from datetime import timedelta
def generate_neg_div_score(df):
    now = datetime.now()
    start_date =  datetime(now.year, now.month, now.day - 3)
    while start_date < now:
        tmp_df = df[df['date'] < start_date].copy()
        generate_defi_neg_div_score(tmp_df, start_date)
        start_date = datetime(start_date.year, start_date.month, start_date.day) + timedelta(days=1)
    

In [91]:
def merge_neg_div_score(df):
    generate_neg_div_score(df)
    neg_div_score_df = pd.read_sql_query('select * from defi_neg_div_score', __connect__())
    neg_div_score_df['timestamp_dt'] = pd.to_datetime(neg_div_score_df['timestamp_dt'])
    if 'price' in neg_div_score_df.columns:
        del neg_div_score_df['price']
    tmp_df_2 = pd.merge(df, neg_div_score_df, how = 'left', left_on=['date', 'Symbol', 'Name'], right_on = ['timestamp_dt','symbol','name'])
    del tmp_df_2['symbol']
    del tmp_df_2['name']
    del tmp_df_2['timestamp_dt']
    return tmp_df_2
    

In [92]:
# df = merge_neg_div_score(df)

In [93]:
# df.columns.tolist()

In [94]:
df

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,HECO Chain Ecosystem,Launchpad,Layer 1,Lending/Borrowing,Options,Polkadot Ecosystem,Solana Ecosystem,Stablecoins,Yield Aggregator,Yield Farming
0,2020-12-21,RARI,Rarible,2.605508,2228569.97,1227740.42,8.553304e+05,NaN,2.500000e+07,NaN,...,,,,,,,,,,
1,2020-12-21,KAVA,Kava.io,1.493707,70019348.83,23278351.50,4.687623e+07,NaN,1.135568e+08,NaN,...,,,,1,,,,,,
2,2020-12-21,RAMP,RAMP,0.038369,4600196.38,300333.82,1.198932e+08,NaN,1.000000e+09,NaN,...,,,,1,,,1,,,
3,2020-12-21,PRQ,PARSIQ,0.408170,42607025.18,4924221.66,1.043856e+08,5.000000e+08,5.000000e+08,NaN,...,,,,,,,,,,
4,2020-12-21,PERP,Perpetual Protocol,1.061385,16946247.63,746906.05,1.596617e+07,NaN,1.500000e+08,NaN,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32193,2021-07-17,POWER,UniPower,0.537834,536893.00,1153.92,9.902500e+05,NaN,1.000000e+06,NaN,...,,,,,,,,,,
32194,2021-07-17,PIPT,Power Index Pool Token,2.700000,2835643.00,32078.00,1.040844e+06,1.040844e+06,1.040844e+06,16552711.0,...,,,,,,,,,,1
32195,2021-07-17,FLUX,Flux,0.089821,16080492.00,115300.00,1.816186e+08,NaN,4.400000e+08,NaN,...,,,,,,,,,,
32196,2021-07-17,STBZ,Stabilize,0.878677,591323.00,35146.00,6.656176e+05,NaN,7.518953e+05,NaN,...,,,,,,,,,,1


## Farms And Pool

### Append the most stable and non stable farm and pool rates.

In [95]:
stable_coins = "USDT, USDC, DAI, BUSD, SUSD, UST, TUSD".split(",")


def get_stable_coin(pair1, pair2):
    if pair1 in stable_coins:
        return pair1
    elif pair2 in stable_coins:
        return pair2
    else:
        return'NA'


def get_pancakswap_and_raydium_farm_data():
    pancakswap_farm_df = pd.read_sql_query('select * from pancakswap_farm_pool_data;', __connect__())
    pancakswap_farm_df = pancakswap_farm_df[['timestamp_dt', 'pair1', 'pair2', 'pool_rate', 'farm_payment', 'where', 'includes_stablecoin']]
    raydium_farm_df = pd.read_sql_query('select * from raydium_farm_pool_data;', __connect__())
    merged_df = pancakswap_farm_df.append(raydium_farm_df)
    return merged_df


def get_cmc_and_liqudityfolio_pool_and_farm_data_df():
    cmc_farm_df = pd.read_sql_query('select * from cmc_farming_yield;', __connect__())
    cmc_farm_df = cmc_farm_df[['timestamp_dt', 'pair1', 'pair2', 'pool_rate', 'farm_payment', 'where', 'includes_stablecoin']]
    cmc_farm_df = cmc_farm_df[cmc_farm_df['where'].isin(['PancakeSwap', 'SushiSwap', 'BakeryToken', 'SUN'])]
    liquidityfolio_pool_df = pd.read_sql_query('select * from liquidityfolio_pool;', __connect__())
    liquidityfolio_pool_df = liquidityfolio_pool_df[liquidityfolio_pool_df['where'] == 'Uniswap']
    liquidityfolio_pool_df = liquidityfolio_pool_df[['timestamp_dt', 'pair1', 'pair2', 'pool_rate', 'farm_payment', 'where', 'includes_stablecoin']]
    liquidityfolio_pool_df['pool_rate'] = liquidityfolio_pool_df['pool_rate'] * 1200
    merged_df = cmc_farm_df.append(liquidityfolio_pool_df)
    return merged_df


def get_farm_and_pool_merged_data_df():
    cmc_and_liqudityfolio_pool_and_farm_data_df = get_cmc_and_liqudityfolio_pool_and_farm_data_df()
    pancakswap_and_raydium_farm_df = get_pancakswap_and_raydium_farm_data()
    merged_df = cmc_and_liqudityfolio_pool_and_farm_data_df.append(pancakswap_and_raydium_farm_df)
    merged_df['pool_rate'] = merged_df['pool_rate']/100
    return merged_df
    

def get_pool_rates(merged_eth_df, stable, defi_symbol):
    data = [np.nan] * 4
    merged_eth_df = merged_eth_df[(merged_eth_df['includes_stablecoin'] == stable) & (merged_eth_df['farm_payment'].isnull())]
    if not merged_eth_df.empty:
        scenario_1 = merged_eth_df.sort_values(['pool_rate']).iloc[-1]
#         print([defi_symbol, merged_eth_df.shape])
        stable_coin = get_stable_coin(scenario_1['pair1'], scenario_1['pair2'])
        other_pair = scenario_1['pair2'] if scenario_1['pair1'] == defi_symbol else scenario_1['pair1']
        data = [other_pair, scenario_1['pool_rate'], stable_coin, scenario_1['where']]
    return data


def get_farm_rates(merged_eth_df, stable, defi_symbol):
    data = [np.nan] * 5
    merged_eth_df = merged_eth_df[(merged_eth_df['includes_stablecoin'] == stable) & (merged_eth_df['farm_payment'].notnull())]
    if not merged_eth_df.empty:
        scenario_1 = merged_eth_df.sort_values(['pool_rate']).iloc[-1]
        stable_coin = get_stable_coin(scenario_1['pair1'], scenario_1['pair2'])
        other_pair = scenario_1['pair2'] if scenario_1['pair1'] == defi_symbol else scenario_1['pair1']
        data = [other_pair, scenario_1['pool_rate'], stable_coin, scenario_1['where'], scenario_1['farm_payment']]
    return data


def get_farm_and_pool_rates(merged_df):
    data = [np.nan] * 18
    defi_symbol = merged_df['defi_symbol'].max()
    merged_eth_df = merged_df[(merged_df['pair1'] == defi_symbol) | (merged_df['pair2'] == defi_symbol)].copy().reset_index(drop=True)
    if not merged_eth_df.empty:
        scenario_1_data = get_pool_rates(merged_eth_df, True, defi_symbol)
        scenario_2_data = get_pool_rates(merged_eth_df, False, defi_symbol)
        scenario_3_data = get_farm_rates(merged_eth_df, True, defi_symbol)
        scenario_4_data = get_farm_rates(merged_eth_df, False, defi_symbol)
        data = []
        data.extend(scenario_1_data)
        data.extend(scenario_2_data)
        data.extend(scenario_3_data)
        data.extend(scenario_4_data)
    r_data = [defi_symbol]
    r_data.extend(data)
    return r_data


def get_stable_non_stable_pool_rates(merged_eth_df):
    grouped_df = merged_eth_df.sort_values('pool_rate').reset_index(drop=True).groupby('timestamp_dt')
    nd_list = grouped_df.apply(lambda x: get_farm_and_pool_rates(x))
    if len(nd_list) > 0:
        index = nd_list.index[0]
        data = [index]
        data.extend(nd_list[0])
        return data
    return None


def get_farm_and_pool_data_symbols(merged_df, defi_symbols):
    data = []
    for defi_symbol in defi_symbols:
        merged_eth_df = merged_df[(merged_df['pair1'] == defi_symbol) | (merged_df['pair2'] == defi_symbol)].copy().reset_index(drop=True)
        merged_eth_df['defi_symbol'] = defi_symbol
        tmp = get_stable_non_stable_pool_rates(merged_eth_df)
        if tmp:
            data.append(tmp)
    return data


def merge_farm_and_pool_data(data_df):
    defi_symbols = data_df['Symbol'].unique().tolist()
    merged_df = get_farm_and_pool_merged_data_df()
    farm_and_pool_data = get_farm_and_pool_data_symbols(merged_df, defi_symbols)
    columns = ['date', 'symbol', 'top_stable_pool_pair', 'top_stable_pool_rate', 'top_stable_pool_coin', 'top_stable_pool_where', 'top_nonstable_pool_pair', 'top_nonstable_pool_rate', 'top_nonstable_pool_coin', 'top_nonstable_pool_where', 'top_stable_farm_pair', 'top_stable_farm_rate', 'top_stable_farm_coin', 'top_stable_farm_where', 'top_stable_farm_reward', 'top_nonstable_farm_pair', 'top_nonstable_farm_rate', 'top_nonstable_farm_coin', 'top_nonstable_farm_where', 'top_nonstable_farm_reward']
    farm_and_pool_df = pd.DataFrame(farm_and_pool_data, columns=columns)
    tmp_df_4 = pd.merge(data_df, farm_and_pool_df, how='left', left_on=['date', 'Symbol'], right_on=['date', 'symbol'])
    del tmp_df_4['symbol']
    tmp_df_5 = tmp_df_4.set_index(tmp_df_4['date'])
    columns = ['top_stable_pool_pair', 'top_stable_pool_rate', 'top_stable_pool_coin', 'top_stable_pool_where', 'top_nonstable_pool_pair', 'top_nonstable_pool_rate', 'top_nonstable_pool_coin', 'top_nonstable_pool_where', 'top_stable_farm_pair', 'top_stable_farm_rate', 'top_stable_farm_coin', 'top_stable_farm_where', 'top_stable_farm_reward', 'top_nonstable_farm_pair', 'top_nonstable_farm_rate', 'top_nonstable_farm_coin', 'top_nonstable_farm_where', 'top_nonstable_farm_reward']
    for column in columns:
        tmp_df_5[column] = tmp_df_5.groupby('Symbol')[column].transform(lambda x: x.ffill())
    tmp_df_5.reset_index(drop=True)
    return tmp_df_5


In [96]:
merged_df_ = get_farm_and_pool_merged_data_df()
merged_df_[merged_df_['where'] == 'balancer']

,timestamp_dt,pair1,pair2,pool_rate,farm_payment,where,includes_stablecoin,tvl


In [97]:
df = merge_farm_and_pool_data(df)
df[df['date'] == pd.to_datetime("4/29/2021")]

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,top_stable_farm_pair,top_stable_farm_rate,top_stable_farm_coin,top_stable_farm_where,top_stable_farm_reward,top_nonstable_farm_pair,top_nonstable_farm_rate,top_nonstable_farm_coin,top_nonstable_farm_where,top_nonstable_farm_reward
date,,,,,,,,,,,,,,,,,,,,,
2021-04-29,2021-04-29,CORE,cVault.finance,6689.235385,6.689235e+07,244088.55,10000.00,NaN,1.000000e+04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-29,2021-04-29,MAPS,MAPS,1.746823,1.091127e+08,1242430.64,62463518.00,1.000000e+10,1.000000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-29,2021-04-29,MOON,MoonSwap,1.047866,1.911968e+07,68132.70,18246296.92,NaN,1.826680e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-29,2021-04-29,REP,Augur,36.215824,3.983741e+08,33445679.30,11000000.00,NaN,1.100000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-29,2021-04-29,FXS,Frax Share,6.869490,6.927374e+07,17450392.11,10084263.31,NaN,9.960156e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-29,2021-04-29,SWISS,swiss.finance,93.846267,0.000000e+00,12270.93,0.00,1.000000e+04,0.000000e+00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-29,2021-04-29,BADGER,Badger DAO,31.672470,2.724844e+08,34546583.31,8603193.57,2.100000e+07,2.100000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-29,2021-04-29,DOUGH,PieDAO DOUGH v2,1.640165,1.114543e+07,1026184.16,6795315.00,NaN,7.672901e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
df[df['date'] == pd.to_datetime("5/1/2021")]['top_nonstable_farm_pair'].unique()

array([nan, 'WBNB', 'YFII', 'ETH', 'BNB'], dtype=object)

In [99]:
# df.columns.tolist()

## Saving to csv File

In [100]:
df.shape

(32198, 74)

In [101]:
df.to_csv('coingecko_data_ratios_and_percentages.csv', index = False)

In [ ]:
df.date.max()

In [ ]:
df[df['Symbol'] == 'AAVE'][['Users life-to-date duneanlytics','date']]

In [ ]:
df

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df.sort_values(['date','Symbol', 'Name'])

(df['Symbol'] + "____" + df['Name']).unique().tolist()

In [ ]:
df[df['Symbol'] == 'BASK']

# QA

In [ ]:
def get_all_mapping_symbols(mapping_table_name, data_table_name, mapping_source_col_name, data_table_col_name):
    sql = f'select * from {mapping_table_name} where symbol != "" and {mapping_source_col_name} in (select distinct {data_table_col_name} from {data_table_name});'
#     print(sql)
    symbols = pd.read_sql_query(sql, __connect__())
    return symbols

def get_datasource_all_tracked_coin_names(data_table_name, data_table_col_name):
    sql = f'select distinct {data_table_col_name} from {data_table_name};'
#     print(sql)
    symbols = pd.read_sql_query(sql, __connect__())
    return symbols[data_table_col_name].values.tolist()

def qa_data_source(df, mapping_table_name, data_table_name, mapping_source_col_name, data_table_col_name):
    source_all_coin_names = get_datasource_all_tracked_coin_names(data_table_name, data_table_col_name)
    print(source_all_coin_names)
    print(len(source_all_coin_names))
    mapping_symbol = get_all_mapping_symbols(mapping_table_name, data_table_name, mapping_source_col_name, data_table_col_name)
    print("Total Mappings: ", mapping_symbol.shape[0])
    name_in_cmc = mapping_symbol['name_in_cmc'].values.tolist()
    captured_names_df = df[df['Name'].isin(name_in_cmc)]
    captured_Names = captured_names_df['Name'].unique().tolist()
    print("Captured Names", len(captured_Names))
    uncaptured_names = [x for x in name_in_cmc if x not in captured_Names]
    print("Uncaptured Names: ", uncaptured_names)
    print(mapping_symbol[mapping_symbol['name_in_cmc'].isin(uncaptured_names)])
    data_symbol_group = captured_names_df.groupby(['Symbol'])
    data_df = data_symbol_group.agg({'date':['max', 'min']}).reset_index()
    print(data_df)
    print("Max Date", data_df['date']['max'].unique())
    

In [ ]:
qa_data_source(df, 'tt_symbols', 'tt_coin_metrics_v2', 'name_in_tt', 'name')

In [ ]:
qa_data_source(df, 'debank_symbols3', 'debank_project_tvl', 'name_in_debank', 'project')

In [ ]:
qa_data_source(df, 'debank_symbols3', 'debank_project_dex', 'name_in_debank', 'project')

In [ ]:
qa_data_source(df, 'debank_symbols3', 'debank_project_lendings', 'name_in_debank', 'symbol')

In [ ]:
qa_data_source(df, 'defi_symbols', 'defi_project_tvl', 'name_in_defi', 'project')

In [ ]:
qa_data_source(df, 'defistation_symbols', 'defistation_data', 'name_in_defistation', 'name')

In [ ]:
qa_data_source(df, 'dune_symbols', 'dune_total_users_analysis2', 'name_in_duneanalytics', 'symbol')

In [ ]:
qa_data_source(df, 'solana_symbols', 'solanaprojects_data', 'name_in_solana', 'name')

In [ ]:
df[df['Symbol'] == 'NMR']